In [1]:

# Imports
import csv
import re
from pathlib import Path

import numpy as np
import pandas as pd

In [3]:
# Project paths

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()

ROOT = PROJECT_ROOT / "data" / "test" / "VICON_CSV"
RESULTS_DIR = PROJECT_ROOT / "results"
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJECT_ROOT)
print("Vicon CSV root:", ROOT)
print("ROOT exists:", ROOT.exists())
print("Results dir:", RESULTS_DIR.resolve())

Project root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2
Vicon CSV root: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/data/test/VICON_CSV
ROOT exists: True
Results dir: /Users/matysprecloux/Desktop/Master IEAP/Code MOTTET/Defense /SYNCOGESTM2/results


In [5]:
def read_vicon_csv(csv_path: Path) -> pd.DataFrame:
    """
    Parse a Vicon "Trajectories" CSV export.

    Expected layout:
    Row 1: "Trajectories"
    Row 2: sampling frequency
    Row 3: marker names (with blanks that must be forward-filled)
    Row 4: axes (Frame/Sub Frame + X/Y/Z)
    Row 5: units (e.g., mm)
    Row 6+: numeric data

    Returns
    -------
    pd.DataFrame
        DataFrame with flattened column names:
        "<marker>_X", "<marker>_Y", "<marker>_Z"
    """
    with open(csv_path, "r", encoding="utf-8", errors="replace", newline="") as f:
        reader = csv.reader(f)
        header_lines = [next(reader) for _ in range(5)]

    marker_row = header_lines[2]
    axis_row   = header_lines[3]

    n = max(len(marker_row), len(axis_row))
    marker_row += [""] * (n - len(marker_row))
    axis_row   += [""] * (n - len(axis_row))

    filled = []
    last = ""
    for m in marker_row:
        m = (m or "").strip()
        if m == "":
            filled.append(last)
        else:
            last = m
            filled.append(last)

    colnames = []
    for m, a in zip(filled, axis_row):
        m = (m or "").strip()
        a = (a or "").strip()

        if a in ["Frame", "Sub Frame"]:
            colnames.append(a)
        elif a in ["X", "Y", "Z"]:
            colnames.append(f"{m}_{a}")
        else:
            colnames.append(m if m else a)

    df = pd.read_csv(csv_path, skiprows=5, header=None, names=colnames, engine="python")
    df = df.dropna(axis=1, how="all")
    return df

In [6]:
def find_xyz_cols(cols, token: str):
    """
    Find X/Y/Z column names for a given marker token, even if prefixed.
    Example: "Patient 1:epaule_D_X"
    """
    pat = re.compile(rf"(?:^|:)\s*{re.escape(token)}_([XYZ])\b", re.IGNORECASE)
    found = {}

    for c in cols:
        c2 = str(c).replace(" ", "")
        m = pat.search(c2)
        if m:
            axis = m.group(1).upper()
            if axis not in found or len(c) < len(found[axis]):
                found[axis] = c

    return found.get("X"), found.get("Y"), found.get("Z")